# Трансферное обучение в TensorFlow

## Задача

Создать свёрточную модель трансферного обучения для классификации бетона с трещиной (Positive mark) и бетона без трещины (Negative mark)  
Как было показано в [2_Models_for_Transfer_Learning_TensorFlow.ipynb](https://github.com/Aleks-Zink/Pet_Projects/blob/main/2_Concrete/2_Models_for_Transfer_Learning_TensorFlow.ipynb), за основу будет взята модель VGG16

## Данные

40 000 цветных картинок бетона размером 227x227 пикселей, 20 000 из которых с стрещиной, другие 20 000 целый  
Данные взяты из курса [AI Capstone Project with Deep Learning](https://www.coursera.org/learn/ai-deep-learning-capstone?specialization=ai-engineer), явлюющийся заключительным курсом [IBM AI Engineering Professional Certificate](https://www.coursera.org/professional-certificates/ai-engineer) на сайте [coursera.org](https://www.coursera.org/)  
[Данные](https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip) 

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip

--2023-04-28 15:24:45--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘concrete_crack_images_for_classification.zip’

concrete_crack_imag 100%[===================>] 233.90M  31.6MB/s    in 7.2s    

2023-04-28 15:24:53 (32.4 MB/s) - ‘concrete_crack_images_for_classification.zip’ saved [245259777/245259777]



In [2]:
!mkdir ./data
!mkdir ./data/concrete
!mkdir ./models

In [ ]:
!unzip concrete_crack_images_for_classification.zip -d ./data/concrete

## Расчёты

In [4]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense

In [5]:
# функция обучения модели
def train(model, train_data, val_data, lr=1e-3, epochs=3, save_model=False):
    # установление оптимизатора, функции ошибки и метрики
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])

    # если save_model=True сохранаяет модель с наилучшей точностью на валидационных данных
    callbacks = None if not save_model else tf.keras.callbacks.ModelCheckpoint(filepath="./models/VGG16_for_Concrete_TensorFlow",
                                                                               monitor="val_accuracy",
                                                                               save_best_only=True,
                                                                               mode='max')

    # обучение модели
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=callbacks)

In [6]:
# инициализация модели

# входной слой модели
i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)

# преобразование данных для данной модели
x = vgg16.preprocess_input(x)

# загрузка предворительно обученной свёрточной модели без полносвязного слоя в конце
x = vgg16.VGG16(include_top=False,
                weights="imagenet",
                pooling="avg")(x)

# добавление полносвязного слоя
x = Dense(units=2, activation="softmax")(x)

# создание модели
model = tf.keras.Model(inputs=i, outputs=x, name="VGG16")

# заморозка весов свёрточной модели
model.layers[-2].trainable = False

model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 tf.cast (TFOpLambda)        (None, None, None, 3)     0         
                                                                 
 tf.__operators__.getitem (S  (None, None, None, 3)    0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, None, None, 3)    0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026   

In [7]:
# загрузка пакетированных данных для теста и валидации
train_data, val_data = image_dataset_from_directory(directory="./data/concrete",
                                                    label_mode="categorical",
                                                    batch_size=100,
                                                    image_size=(224,224),
                                                    shuffle=True,
                                                    seed=0,
                                                    validation_split=0.25,
                                                    subset="both",
                                                    interpolation="bilinear")

Found 40000 files belonging to 2 classes.
Using 30000 files for training.
Using 10000 files for validation.


In [8]:
# обучение модели
train(model=model, train_data=train_data, val_data=val_data, lr=1e-3, epochs=3, save_model=False)

Epoch 1/3
300/300 [==============================] - 184s 547ms/step - loss: 0.1253 - accuracy: 0.9556 - val_loss: 0.0291 - val_accuracy: 0.9928
Epoch 2/3
300/300 [==============================] - 174s 577ms/step - loss: 0.0207 - accuracy: 0.9956 - val_loss: 0.0171 - val_accuracy: 0.9951
Epoch 3/3
300/300 [==============================] - 178s 591ms/step - loss: 0.0134 - accuracy: 0.9968 - val_loss: 0.0128 - val_accuracy: 0.9963


Для повышения точности разморозим все веса модели и дообучим всю модель с уменьшенной скоростью, для подстраивания свёрточных слоёв под наши данные

In [9]:
# разморозка весов
model.trainable = True

model.summary()

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 tf.cast (TFOpLambda)        (None, None, None, 3)     0         
                                                                 
 tf.__operators__.getitem (S  (None, None, None, 3)    0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, None, None, 3)    0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                             

In [10]:
# обучение модели c уменьшенной скоростью обучения
train(model=model, train_data=train_data, val_data=val_data, lr=1e-4, epochs=3, save_model=True)

Epoch 1/3
300/300 [==============================] - ETA: 0s - loss: 0.0748 - accuracy: 0.9921

300/300 [==============================] - 470s 1s/step - loss: 0.0748 - accuracy: 0.9921 - val_loss: 0.0088 - val_accuracy: 0.9978
Epoch 2/3
300/300 [==============================] - ETA: 0s - loss: 0.0076 - accuracy: 0.9977

300/300 [==============================] - 440s 1s/step - loss: 0.0076 - accuracy: 0.9977 - val_loss: 0.0066 - val_accuracy: 0.9983
Epoch 3/3
300/300 [==============================] - 438s 1s/step - loss: 0.0058 - accuracy: 0.9987 - val_loss: 0.0076 - val_accuracy: 0.9980


In [11]:
# загружаем модель с наилучшей точностью
model = tf.keras.models.load_model('./models/VGG16_for_Concrete_TensorFlow')

print(f"Получаем модель с:\n\tОбъёмом {model.count_params() * 4 / (1024 ** 2):.2f} Мб\n\tТочностью {model.evaluate(val_data)[1]*100:.2f}%")

100/100 [==============================] - 44s 438ms/step - loss: 0.0066 - accuracy: 0.9983
Получаем модель с:
	Объёмом 56.14 Мб
	Точностью 99.83%


Для дальнейшего использования модели, входные данные должны иметь форму (B, H, W, C)  
Где:
 - B: количество фото для анализа
 - H = 224: высота фото
 - W = 224: ширина фото
 - С = 3: цветовых каналов

## Результаты

Была получена свёрточная модель трансферного обучения на базе модели VGG16 с размером модели в ~56 Мб.  
Получившаяся модель с точностью ~99.83% может классифицировать фотографии бетона с трещиной и бетона без трещины

## Опыт работы в TensorFlow

Плюсы:
 + Большое количество инструментов 
 + Простой высокоуровневый API Keras 
 + Удобная реализация autograd
 
Минусы:
 + Много вещей скрыто за высокоуровневым API, что приводит к путанице
 + Модель сохраняемая TensorFlow весит в разы больше, чем её расчётный / теоретический объём